In [ ]:
#Logan Kelsch + JJ
'''
/*
 *  taking this version of neural network training takes only a few features
 *          [ high, low, open, close, volume, TimeOfDay, DayOfWeek ]
 *  and uses common techniques described in 'features_creation.py' to largely
 *  expand dimensionality. 
 *  As of writing this (11/24/24) there is limited expansion
*/
''' 

#IMPORT LIBRARIES-------------------------------------------------------

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
from keras.initializers import GlorotUniform
from keras.initializers import RandomNormal
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from funcs_data_preprocess import *
from feature_creation import *


# Load the dataset
data = pd.read_csv('betaset_tmp.csv')

#to_drop = fn_hilo_diff()

#data = data.drop(columns=to_drop)



indp_size       = 0.1   ###
test_size       = 0.2

num_isol_feats  = 0
use_PCA         = True
comps_PCA       = 32
target          = 15
time_steps      = 15
#normalized ToD values
t_start         = 570
t_end           = 645


indp_splt = int(len(data)*(1-indp_size))

print(f"\nIndependent data size: {len(data.iloc[indp_splt:])}\n")

#drop unused target columns
data = data.drop(columns=[f't_{i}' for i in range(1,target)])
data = data.drop(columns=[f't_{i}' for i in range(target+1,61)])

#collect all sample indices to be kept through time filter
#two seperate functions to keep main and ind test isolated
data_keep_ndx = grab_wanted_times(data.values, t_start, t_end, time_steps)
main_keep_ndx = [n for n in data_keep_ndx if n < indp_splt]
indp_keep_ndx = [n for n in data_keep_ndx if n >= indp_splt]
indp_keep_ndx = [(n - indp_splt) for n in indp_keep_ndx]

data = data.drop(columns=fn_orig_price())
data = data.drop(columns=fn_orig_time())

#split dependent and independent
data_independent = data.iloc[indp_splt:]
data = data.iloc[:indp_splt]

#confirmation printout of all features/targets
Xfeatures = data.columns[:-1]
Yfeatures = data.columns[-1]
print("TESTED FEATURES: ")
print(Xfeatures)
print("TESTING FOR: ")
print(Yfeatures)


# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

#do the same for independent set
X_ind = data_independent.iloc[:, :-1].values
y_ind = data_independent.iloc[:, -1].values

X_nrm = normalize_from_tt_split(X,     X, test_size)
X_ind = normalize_from_tt_split(X_ind, X, test_size)
if(use_PCA):
    X     = reform_with_PCA_isolated(X_nrm, X_nrm, test_size, num_isol_feats, comps_PCA)
    X_ind = reform_with_PCA_isolated(X_ind, X_nrm, test_size, num_isol_feats, comps_PCA)
X,      y       = reformat_to_lstm(X,       y,      time_steps)
X_ind,  y_ind   = reformat_to_lstm(X_ind,   y_ind,  time_steps)

print('\nX shape == {}.'.format(X.shape))
print('y shape == {}.\n'.format(y.shape))


print(f'Raw Sample Count:\t{len(X)}')
X,     y     = filter_times(X,     y,     main_keep_ndx)
X_ind, y_ind = filter_times(X_ind, y_ind, indp_keep_ndx)
print(f'Remaining Sample Count:\t{len(X)}')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

#to remove overlapping values that were left from training set
X_test = X_test[time_steps:]
y_test = y_test[time_steps:]

print('\nX_train shape == {}.'.format(X_train.shape))
print('y_train shape == {}.'.format(y_train.shape))
print('X_test shape == {}.'.format(X_test.shape))
print('y_test shape == {}.\n'.format(y_test.shape))


#LEARNING RATES____________________________________________________________________________________________
from keras.optimizers.schedules import ExponentialDecay
lr_schedule = ExponentialDecay(
    #good rough val to start, .25, good val to end at .0015.
    #5k epoch should be: .25, 8565, .9995, true
    0.01,
    decay_steps=100,
    decay_rate=0.997,
    staircase=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.75, 
    patience=12, 
    min_lr=1e-8
)

opt1 = SGD(learning_rate=0.01)
opt2  = tf.keras.optimizers.Adam(learning_rate=0.001)
opt3 = SGD(learning_rate=lr_schedule)
opt4 = SGD(learning_rate=0.0001, momentum=0.9)
opt5 = tf.keras.optimizers.Adam(learning_rate=0.001)

#BUILD AND LOAD MODEL__________________________________________________________________________________________


#print(X_train.shape[0]/time_steps)
#X_train = np.reshape(X_train,((X_train.shape[0]//time_steps), time_steps, 35))  # Reshape to (batch_size, 5 time steps, 35 features)
#y_train = y.reshape(1,-1)
from tensorflow.keras import regularizers

def build_LSTM_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],X_train.shape[2])),
        tf.keras.layers.LSTM(128, activation='tanh', recurrent_dropout=0.2, return_sequences=True),#, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(128, activation='tanh', recurrent_dropout=0.25, return_sequences=False),#, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        #tf.keras.layers.Dropout(0.2),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Dense(16),# kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Activation('relu'),
        #tf.keras.layers.Dropout(0.0),
        #tf.keras.layers.Dense(8),# kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        #tf.keras.layers.BatchNormalization(),
        #tf.keras.layers.Activation('relu'),
        #tf.keras.layers.Dropout(0.0),
        tf.keras.layers.Dense(1, activation='linear')
    ])
    model.compile(optimizer=opt4,
                  loss='mse'
                  ,metrics=['R2Score','root_mean_squared_error'])
    return model

def load_model():
    #loaded_model = tf.keras.models.load_model('tupleTrain.keras', custom_objects={'custom_loss':custom_loss})
    loaded_model = tf.keras.models.load_model('r30_10s_LSTM_8.keras')
    loaded_model.compile(optimizer=opt4,
                         loss='mse'
                         , metrics=['R2Score','root_mean_squared_error'])
    return loaded_model

#TRAIN THE MODEL WITH CUSTOMIZABLE EPOCHS-------------------------------------------------------

epochs = 20

early_stopping = EarlyStopping(monitor='val_loss', patience=8, mode='min', restore_best_weights=True)

cmp = 'C'
if tf.config.list_physical_devices('GPU'):
    cmp = 'G'
    pass
with tf.device('/'+cmp+'PU:0'):
    print('Running on: '+cmp+'PU\n')
    model = build_LSTM_model()
    #loaded_model = load_model()
    used_model = model
    history = used_model.fit(X_train, y_train, epochs=epochs,\
                        shuffle=False, verbose=1, validation_data=(X_test, y_test),\
                        batch_size=120,callbacks=[reduce_lr, early_stopping])

#EVALUATE THE MODEL AND VISUALIZE RESULTS-------------------------------------------------------

#show_all_results(used_model, history, X_test, y_test)

# LOSS
epochs = range(1, len(history.history['loss']) + 1)
plt.figure(figsize=(12, 6))
plt.plot(epochs[1:], history.history['loss'][1:], 'y', label='Training Loss')
plt.plot(epochs[1:], history.history['val_loss'][1:], 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()
# R2SCORE

plt.plot(epochs[1:], history.history['R2Score'][1:], 'y', label='Training R2')
plt.plot(epochs[1:], history.history['val_R2Score'][1:], 'r', label='Validation R2')
plt.title('Training and Validation R2Score')
plt.xlabel('Epoch')
plt.ylabel('R2Score')
plt.ylim(bottom=0)
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()


#predicting the test set results
y_pred = used_model.predict(X_test) 


plt.scatter(y_pred, y_test, s=1)
plt.axis('tight')
plt.title('Testing Outputs')
plt.xlabel('y_pred')
plt.xlim(-25,25)
plt.ylim(-25,25)
plt.ylabel('y_test')
ax = plt.gca()
x_vals = np.array(ax.get_xlim())
y_vals = x_vals  # Since y = x
plt.plot(x_vals, y_vals, '-', color='black', label='y = x', linewidth=0.5)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()

#SCATTERPLOT #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  
plt.scatter(y_pred, y_test, s=1)
plt.grid()
plt.axis('tight')
plt.title('Testing Outputs')
plt.xlabel('y_pred')
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.ylabel('y_test')
ax = plt.gca()
x_vals = np.array(ax.get_xlim())
y_vals = x_vals  # Since y = x
plt.plot(x_vals, y_vals, '-', color='black', label='y = x', linewidth=0.5)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()

used_model.save('tmp.keras')

In [7]:
used_model.save('r30_PCA10_6.keras')

In [ ]:
#new_model = tf.keras.models.load_model("r30_PCA10_1.keras")

X_exam = X_ind
y_exam = y_ind

y_pred = used_model.predict(X_exam) 

'''
plt.figure(figsize=(7, 6))
#SCATTERPLOT #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  #SCATTERPLOT  
plt.scatter(y_pred, y_exam, s=1)
plt.grid()
plt.axis('tight')
plt.title('Testing Outputs')
plt.xlabel('y_pred')
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.ylabel('y_test')
ax = plt.gca()
x_vals = np.array(ax.get_xlim())
y_vals = x_vals  # Since y = x
plt.plot(x_vals, y_vals, '-', color='black', label='y = x', linewidth=0.5)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()
'''
s_kws = {'s':1,'color':(0,.35,0)}
l_kws = {'lw':1,'color':(0,.35,0)}

import seaborn as sns

#Training set
y_pred = used_model.predict(X_train) 
#sns.set_theme(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme()
y_pred = np.squeeze(y_pred)
y = np.squeeze(y)
ys = pd.DataFrame({"y_pred":y_pred,"y_true":y_train})
g = sns.lmplot(data=ys,x="y_pred", y="y_true", scatter_kws=s_kws, line_kws=l_kws)#,hue="MO")
g.fig.set_size_inches(7,6)
plt.title('Training Data Performance')
plt.grid()
y_min, y_max = ys['y_true'].min(), ys['y_true'].max()
plt.ylim(y_min, y_max)
x_center = np.mean(ys['y_pred'])
y_range = y_max - y_min
x_min = x_center - y_range / 2
x_max = x_center + y_range / 2
plt.xlim(x_min, x_max)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()

#Validation set
y_pred = used_model.predict(X_test) 
#sns.set_theme(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme()
y_pred = np.squeeze(y_pred)
y = np.squeeze(y)
ys = pd.DataFrame({"y_pred":y_pred,"y_true":y_test})
g = sns.lmplot(data=ys,x="y_pred", y="y_true", scatter_kws=s_kws, line_kws=l_kws)#,hue="MO")
g.fig.set_size_inches(7,6)
plt.title('Validation Data Performance')
plt.grid()
y_min, y_max = ys['y_true'].min(), ys['y_true'].max()
plt.ylim(y_min, y_max)
x_center = np.mean(ys['y_pred'])
y_range = y_max - y_min
x_min = x_center - y_range / 2
x_max = x_center + y_range / 2
plt.xlim(x_min, x_max)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()


#independant data performance
y_pred = used_model.predict(X_exam) 
#sns.set_theme(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme()
y_pred = np.squeeze(y_pred)
y = np.squeeze(y)
ys = pd.DataFrame({"y_pred":y_pred,"y_true":y_exam})
g = sns.lmplot(data=ys,x="y_pred", y="y_true", scatter_kws=s_kws, line_kws=l_kws)#,hue="MO")
g.fig.set_size_inches(7,6)
plt.title('Independent Data Performance')
plt.grid()
y_min, y_max = ys['y_true'].min(), ys['y_true'].max()
plt.ylim(y_min, y_max)
x_center = np.mean(ys['y_pred'])
y_range = y_max - y_min
x_min = x_center - y_range / 2
x_max = x_center + y_range / 2
plt.xlim(x_min, x_max)
plt.axhline(0, color='black', linewidth=0.5)
plt.axvline(0,color='black',linewidth=0.5)
plt.show()

In [ ]:
epochs = range(1, len(history.history['loss']) + 1)
plt.figure(figsize=(12, 6))
plt.plot(epochs[1:], history.history['loss'][1:], 'y', label='Training Loss')
plt.plot(epochs[1:], history.history['val_loss'][1:], 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()